In [1]:
import numpy as np
import numpy.random as random

In [22]:
import importlib

In [26]:
import board

In [30]:
importlib.reload(board);

In [31]:
B = board.Board(n=3)
B.movesToBoard([ (1,[0,0]), (1,[0,1]), (2,[0,2])])
B.printBoard()

    0 1 2 
0  |X X 0 |
1  |      |
2  |      |
Winner: -1


In [5]:
def simulateGame(B, p1=None, p2=None, rnd=0, playerToMove_ = 1, max_moves = 10000):
    history = []
    board = B.board
    masks = B.masks
    n_moves = 0
    playerToMove = playerToMove_
    while B.getWinner() == -1 and n_moves<max_moves:
        n_moves = n_moves+1
        # Chose a move (random or use a player model if provided)
        move = None
        moves = B.getMoves()
        move = moves[random.randint(0, len(moves) )]
       
        # Make the move
        B.board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

In [21]:
B = Board(5)
simulateGame(B);
B.printBoard()

    0 1 2 3 4 
0  |X O X O   |
1  |X X   O O |
2  |O O X X X |
3  |O O X X X |
4  |  O O   X |
Winner: 1
